### The Topology

Prior to initiating any tests then the network needs to be described in a Topology File. 


The topology file is written in yaml and will describe attributes of your test network, 
such as device type, login details, login method, links between devices etc.  

The GENIE Topology object is created by issuing the following commands:-

In [ ]:
from genie.conf import Genie

testbed = Genie.init('../scripts/vagrant_single_ios.yaml')

The topology object that has been created is called testbed.  Now look at some of the attributes
of the topology object by issuing the following commands

In [ ]:
testbed.devices 

testbed.name 

testbed.interfaces

The topology file has numerous attributes, objects and methods, to view these then in the iPython session type
**testbed.** and press tab.  Alternatively you can issue the following command within iPython

In [ ]:
dir(testbed)

To explore further attributes and methods then enter the following within iPython (or tab completion)

In [ ]:
dir(testbed.devices)

You will see that from issuing this command that testbed.devices has number of attributes/objects/methods, of note are
_iosxe1_ and _iosxe2_

You should see are large number of attributes/objects/methods.  Of particular note is the _connect_ method.  We shall be using the 
_connect_ method to establishing connectivity with our testbed devices in forthcoming exercises.

In [ ]:
dir(testbed.devices.iosxe1)

### GENIE OPS Library


The GENIE OPS Library is used to represent a device/feature's operational state/data through a Python Object. 
Each feature on each device is represented via a single Ops object instance, where state/status 
information is stored as an object attribute.

Ops objects are snapshots of a particular feature on a particular device at a specific time.
 
To demonstrate the power of the GENIE OPS library then please follow the sections below.

To start make sure that your Python Virtual Environment is still running from step 2 and that you are in 
the scripts directory.
Initiate an iPython interactive session

In [ ]:
from pprint import pprint
from genie.conf import Genie

from genie.libs.ops.interface.iosxe.interface import Interface

testbed = Genie.init('../scripts/vagrant_single_ios.yaml')

The commands above will:-

* Import the pprint library so as to 'pretty print' structured data to make it more easily readable
* From genie.conf import the Genie Class
* Import the Operational Model for IOSXE Interfaces
* Initiate the testbed file in order to interact with the testbed devices


Access to the devices needs to be established prior to sending any additional GENIE API calls to the device, leveraging
the topology _connect_ method. 

First make a reference to the topology device object

In [ ]:
uut = testbed.devices.iosxe1

As mentioned previously the device object has a method called connect.  Using the connect method will establish a connection to the device
using the connection method described in the topology yaml file, in this case _ssh_.  You will know that a connection is successful with the 
output from the device being displayed in the interactive session. Once connection is made the device will be prepared 
for further calls on the device.

In [ ]:
uut.connect()

### Learn the state of the interfaces on the device under test (iosxe1)

First an interface Ops object needs to instantiated.  The argument for instantiating the object is the device that is
being tested, defined earlier as _uut_.

In [ ]:
interface = Interface(device=uut)

The _interface_ object that has been instantiated has a **learn** method.  The learn method will send several 
relevant show commands to an IOSXE device.  The output of the show commands will be parsed and collated and subsequently stored
as a single structured data entity(dictionary).

In [ ]:
interface.learn()

The data that is parsed and collated is stored as a single entry under the _info_ attribute of the interface object.
  
To view all the returned data:-

In [ ]:
pprint(interface.info)

From the above output you should recognise that the data is now stored as a dictionary and thus the values can be 
retrieved by referencing the relevant key.

For example:-

In [ ]:
pprint(interface.info['nve1'])

And:-
    

In [ ]:
pprint(interface.info['nve1']['phys_address'])

**Partial retrieval of Ops data**

Rather than retrieving the entire state you can choose to only save the attributes you require for the interface.  
For example we only wish to retrieve the Mac Addresses of the interfaces.  To achieve this

In [ ]:
interface = Interface(device=uut, attributes=['info[(.*)][mac_address]'])

Now 'relearn' the interface object and display the output

In [ ]:
interface.learn()

In [ ]:
pprint(interface.info)

**Verify State**

A very useful feature of the Ops object is to verify the condition of a particular state.  

The code below creates a function that checks the current oper_status of GigabitEthernet3.  

If the oper_status is up, then the verification is successful it will print that Gig3 is up and return to the main body of the
code.  

If the oper_status is down it will learn the interface state 3 more times with a sleep interval of 3 seconds, if after 3 attempts
the interface is still down then an Exception will be raised.

Enter the code as is below to your iPython session


In [ ]:
interface = Interface(device=uut)

def verify_interface_status(obj):
    if obj.info['GigabitEthernet3'].get('oper_status', None) and\
       obj.info['GigabitEthernet3']['oper_status'] == 'up':
       print('\n\nGig 3 is up')
       return
    raise Exception('Gig 3 is currently down')
    
interface.learn_poll(verify=verify_interface_status, sleep=3, attempt=3)

If the lab you are using is the Sandbox - ssh cisco@10.10.20.48   password cisco_1234!  

If the lab you are using is a local vagrant machine - ssh -p 3122 vagrant@127.0.0.1 vagrant

and...

conf t  
interface GigabitEthernet3  
shutdown

In [ ]:
interface = Interface(device=uut)

def verify_interface_status(obj):
    if obj.info['GigabitEthernet3'].get('oper_status', None) and\
       obj.info['GigabitEthernet3']['oper_status'] == 'up':
       print('\n\nGig 3 is up')
       return
    raise Exception('Gig 3 is currently down')
    
interface.learn_poll(verify=verify_interface_status, sleep=3, attempt=3)


**Compare State**

In order to determine what state has changed over time we can compare 'snapshots'.  Consider that each time you 
initiate the learn method, you are effectively taking a snapshot of current state.  

The code below will demonstrate, please enter into iPython:-

In [ ]:
interface_after = Interface(device=uut, attributes=['info[(.*)][bandwidth]'])
interface_after.learn()

Now in a seperate terminal ssh into your device

If the lab you are using is the Sandbox - ssh cisco@10.10.20.48   password cisco_1234!  

If the lab you are using is a local vagrant machine - ssh -p 3122 vagrant@127.0.0.1 vagrant

and......

conf t  
interface GigabitEthernet3  
bandwidth 50000

Now enter the following code:-

In [ ]:
interface_before = Interface(device=uut, attributes=['info[(.*)][bandwidth]'])
interface_before.learn()

And finally compare the two by entering the following code:-

In [ ]:
diff = interface_after.diff(interface_before)
print(diff)

Disconnect from the device

In [ ]:
uut.disconnect()

**Conclusion**

As demonstrated the Ops library is an extremely useful set of tools for retrieving state data from your devices.  The
preceding exercise only explored the Ops _Model_ for IOSXE Interfaces.  There are hundreds of further models at your disposal
that support a vast range of features across IOSXE, IOSXR and NXOS.  To view the available models please go to [Model Wiki](https://pubhub.devnetcloud.com/media/pyats-packages/docs/genie/genie_libs/#/models)
